In [1]:
# connect to the device
from thorlabs_apt_device.devices.kdc101 import KDC101
from thorlabs_apt_device.protocol import *
'''
"rotationStage": {
            "managerName": "KDC101Manager",
            "managerProperties": {
                "port": "COM12",
                "units": "degrees",
                "posConvFac": 1919.641857862339,
                "velConvFac": 42941.66,
                "accConvFac": 14.66
            }
        }
https://www.thorlabs.com/Software/Motion%20Control/APT_Communications_Protocol.pdf

Sample stage S/N 27266343 Port 14 or 15
Camera stage S/N 27266327 Port 14 or 15
Actuator Z8235B  EncCnt per mm  34554.96  Velocity 772981.3692 (mm/s)  Acceleration 263.8443072 (mm/s2)
APT communication protocol page 39

Triggering the stage, information is in the APT communication protocol page 140

widget from abigail
in SetupStatusWidget:
"""Rotation stage"""
self.rotationStageHeader = QtWidgets.QLabel('Rotation stage')
self.rotationStageHeader.setFont(QtGui.QFont('Calibri', 14))
self.rotationStageHeader.setStyleSheet("font-weight: bold")

self.rotationStagePosLabel = QtWidgets.QLabel('Set position (deg) of rotation stage')
self.rotationStagePosEdit = guitools.BetterDoubleSpinBox()
self.rotationStagePosEdit.setDecimals(4)
self.rotationStagePosEdit.setMinimum(-360)
self.rotationStagePosEdit.setMaximum(360)
self.rotationStagePosEdit.setSingleStep(0.01)
self.jogStepSizeLabel = QtWidgets.QLabel('Set jog step size (deg)')
self.jogStepSizeEdit = guitools.BetterDoubleSpinBox()
self.jogStepSizeEdit.setDecimals(4)
self.jogStepSizeEdit.setMaximum(360)
self.jogStepSizeEdit.setSingleStep(0.01)
self.jogPositiveButton = guitools.BetterPushButton('>>')
self.jogNegativeButton = guitools.BetterPushButton('<<')
self.currentPosOfRotationStageLabel = QtWidgets.QLabel('Current position (deg) of rotation stage')
self.currentPosOfRotationStageDisp = QtWidgets.QLabel('')

'''
import time
manager_properties = {
    "port": "COM14",
    "units": "mm",
    "posConvFac": 34554.96,
    "velConvFac": 772981.3692,
    "accConvFac": 263.8443072
}

stage = KDC101(serial_port="COM14")
time.sleep(5)
stage.set_jog_params(int(0.2*manager_properties['posConvFac']),
                     int(2*manager_properties['accConvFac']), 
                     int(1.5*manager_properties["velConvFac"]))
stage.set_velocity_params(int(2*manager_properties["accConvFac"]),
                          int(1.5*manager_properties["velConvFac"]))

In [2]:
# get trigger info
stage.move_absolute(100000)

In [16]:
stage.set_moverelparams(relative_distance=4000)

In [2]:
stage.trigger_params

{'trig1_mode': '',
 'trig1_polarity': '',
 'trig2_mode': '',
 'trig2_polarity': '',
 'msg': '',
 'msgid': 0,
 'source': 0,
 'dest': 0,
 'chan_ident': 0}

In [3]:
for i in range(100):
    time.sleep(0.5)
    print(stage.status['position'])

100000
100000
100000
100000


KeyboardInterrupt: 

In [2]:
# 2.
stage.velparams

{'min_velocity': 0,
 'max_velocity': 0,
 'acceleration': 0,
 'msg': '',
 'msgid': 0,
 'source': 0,
 'dest': 0,
 'chan_ident': 0}

In [2]:
# set trigger params
stage.set_triggerparams(
    trig1_mode=0,
    trig1_polarity=1,
    trig2_mode=0,
    trig2_polarity=1
    )

In [9]:
from typing import Optional, Sequence
import struct


def _pack(
    msgid: int,
    dest: int,
    source: int,
    *,
    param1: int = 0,
    param2: int = 0,
    data: Optional[bytes] = None
):
    if data is not None:
        assert param1 == param2 == 0
        return struct.pack("<HHBB", msgid, len(data), dest | 0x80, source) + data
    else:
        return struct.pack("<H2b2B", msgid, param1, param2, dest, source)


def mot_get_kcubetrigconfig(dest: int, source: int, chan_ident: int):
    return _pack(0x0525, dest, source, param1=chan_ident)

get_response = mot_get_kcubetrigconfig(stage.status['dest'],
                        stage.status['source'],
                        stage.status['chan_ident'])
print(get_response)

b'%\x05\x01\x00\x01P'


MGMSG_MOT_SET_KCUBETRIGIOCONFIG 0x0523
MGMSG_MOT_REQ_KCUBETRIGIOCONFIG 0x0524
MGMSG_MOT_GET_KCUBETRIGIOCONFIG 0x0525

MGMSG_MOT_SET_KCUBETRIGIOCONFIG 0x0523
MGMSG_MOT_REQ_KCUBETRIGIOCONFIG 0x0524
MGMSG_MOT_GET_KCUBETRIGIOCONFIG 0x0525

MGMSG_MOT_SET_KCUBEPOSTRIGPARAMS 0x0526
MGMSG_MOT_REQ_KCUBEPOSTRIGPARAMS 0x0527
MGMSG_MOT_GET_KCUBEPOSTRIGPARAMS 0x0528

In [10]:
print(stage.request_kcube_trigger_config(1))
print(stage.get_kcube_trigger_config())

None


TypeError: 'NoneType' object is not subscriptable

In [5]:
import os
import fnmatch
import re

def search_string_in_files(directory, search_string):
    matches = []
    pattern = re.compile(search_string)

    for root, dirnames, filenames in os.walk(directory):
        for filename in fnmatch.filter(filenames, '*'):
            filepath = os.path.join(root, filename)
            try:
                with open(filepath, 'r', encoding='utf-8', errors='ignore') as file:
                    for line_number, line in enumerate(file, 1):
                        if pattern.search(line):
                            matches.append({
                                'file': filepath,
                                'line_number': line_number,
                                'line': line.strip()
                            })
            except Exception as e:
                print(f"Could not read file {filepath}: {e}")

    return matches

# Example usage
directory_to_search = 'C:/Users/alm/git/ImSwitchConfig/'
directory_to_search = 'C:/Users/alm/AppData/Roaming/Python/Python310/site-packages/thorlabs_apt_device/'
print(directory_to_search)
string_to_search = 'mot_get_homeparams'
results = search_string_in_files(directory_to_search, string_to_search)

for match in results:
    print(f"Found in {match['file']} on line {match['line_number']}: {match['line']}")


C:/Users/alm/AppData/Roaming/Python/Python310/site-packages/thorlabs_apt_device/
Found in C:/Users/alm/AppData/Roaming/Python/Python310/site-packages/thorlabs_apt_device/devices\aptdevice_motor.py on line 255: "mot_get_homeparams",
Found in C:/Users/alm/AppData/Roaming/Python/Python310/site-packages/thorlabs_apt_device/devices\aptdevice_motor.py on line 307: elif m.msg == "mot_get_homeparams":
Found in C:/Users/alm/AppData/Roaming/Python/Python310/site-packages/thorlabs_apt_device/devices\__pycache__\aptdevice_motor.cpython-310.pyc on line 116: mot_get_statusupdatemot_get_dcstatusupdatemot_move_stoppedmot_move_completedmot_get_velparamsmot_get_genmoveparamsmot_genmoveparamsmot_get_jogparamsmot_get_homeparamsmot_get_avmodesr  Message  has unrecognised source=. has unrecognised channel=)rf   rg   rh   ri   r   r   r   r   rj   )rk   rl   rm   rn   )rJ   _process_messager,   r.      USBr   index
Found in C:/Users/alm/AppData/Roaming/Python/Python310/site-packages/